In [2]:
import pandas as pd

# Load dataset
df = pd.read_parquet("data/tron_blockchain_interview_data.parquet")

# Convert timestamp
df['datetime'] = pd.to_datetime(df['unixtimestamp'], unit='s')

# Preview data
df.head()

,unixtimestamp,transaction,fromAddress,toAddress,token,token_amount,usd_value,datetime
0,1735742709,8a8b7e3c51f1e15e8fb77d38e5f44fcc319a2053bee4d5...,TJ7hhYhVhaxNx6BPyq7yFpqZrQULL3JSdb,TPq6UAn94sKqVmtxybdMemVWiDcpQhAx43,USDT,846000000,844.310401,2025-01-01 14:45:09
1,1735742709,757cdf71a34741a5a6e1c085dd2408e4f6f49a67a77a6b...,TEmb2qSrWtmsx4F8aijGAy3CRApKLx3JtT,TCZ4tPauSPj6bAoYbJYq8TQvaYziXiZqLS,USDT,279000000,278.442792,2025-01-01 14:45:09
2,1735742709,12aa49b2915a33f37da62b1c04451cc96c3aeea7bbbc1c...,TMQkP3upAn4reUUw5HeXVWy6PjhWw3kVbz,TWAudzu86TJ8DtHb4wBvxWx2EpmqtcD3Wb,TRX,4300000,1.089288,2025-01-01 14:45:09
3,1735742709,faa2668b2864f2934b210c30bb1dd18d01c5eb95721732...,TLj5jNLzaoR94c5WUH9uxpQAY3RZT6gh2y,TWg8JWoB646F6DRAU8vkKMUsSiKa4htV2v,USDT,100166300,99.966252,2025-01-01 14:45:09
4,1735742709,8dcbb9d945567e210ff8d86e8157c5fc75ca0a557d6568...,TL6h4ANWARrjTnSK1HYNfDc9m5HV9GDKoZ,TTCtwXVkHwmcEQ9od7aW8WMp72dmzxMFhB,USDT,52460000,52.355229,2025-01-01 14:45:09
